In [25]:
import json 
import pandas as pd
import numpy as np 
import os
import ast


In [32]:
base_dir = "/Users/theo/Documents/random/datadiop/data"
seasons = [f"{y}-{str(y+1)[-2:]}" for y in range(2018, 2025)]

# Load id_dict for FPL player mapping
id_dict_path = os.path.join(base_dir, "2021-22", "id_dict.csv")
id_dict = pd.read_csv(id_dict_path)
id_dict = id_dict.rename(columns=lambda x: x.strip())
id_dict["FPL_ID"] = id_dict["FPL_ID"].astype(str)

season_dbs = {}

for season in seasons:
    gws_dir = os.path.join(base_dir, season, "gws")
    teams_path = os.path.join(base_dir, season, "teams.csv")
    fixtures_path = os.path.join(base_dir, season, "fixtures.csv")
    if not os.path.isdir(gws_dir) or not os.path.isfile(teams_path) or not os.path.isfile(fixtures_path):
        continue

    teams = pd.read_csv(teams_path)
    teams = teams.rename(columns={'id': 'team_id'})
    fixtures = pd.read_csv(fixtures_path)
    # Map team ids to names
    team_id_to_name = teams.set_index('team_id')['name'].to_dict()
    team_id_to_short = teams.set_index('team_id')['short_name'].to_dict()
    fixtures['team_h_name'] = fixtures['team_h'].map(team_id_to_name)
    fixtures['team_a_name'] = fixtures['team_a'].map(team_id_to_name)
    fixtures['team_h_short'] = fixtures['team_h'].map(team_id_to_short)
    fixtures['team_a_short'] = fixtures['team_a'].map(team_id_to_short)

    gw_dfs = []
    for fname in sorted(os.listdir(gws_dir)):
        if fname.startswith("gw") and fname.endswith(".csv"):
            gw_num = fname[2:-4]
            fpath = os.path.join(gws_dir, fname)
            try:
                df = pd.read_csv(fpath, encoding="utf-8")
            except UnicodeDecodeError:
                df = pd.read_csv(fpath, encoding="latin1")
            df["gw"] = gw_num
            df["season"] = season

            # After merging GW with fixtures
            df = df.merge(
                fixtures[["id", "team_h", "team_a", "team_h_name", "team_a_name", "team_h_score", "team_a_score"]],
                left_on="fixture",
                right_on="id",
                how="left"
            )

            # Infer player's team name
            df["player_team"] = df.apply(
                lambda row: row["team_h_name"] if row["was_home"] else row["team_a_name"], axis=1
            )

            # Optionally, merge team info for player's team
            df = df.merge(
                teams[["team_id", "name", "short_name"]],
                left_on="player_team",
                right_on="name",
                how="left",
                suffixes=("", "_playerteam")
            )
            gw_dfs.append(df)
if gw_dfs:
    season_dbs[season] = pd.concat(gw_dfs, ignore_index=True)

# Example: access 2018-19 season DataFrame
# df_2018_19 = season_dbs["2018-19"]

In [39]:
season_dbs['2024-25'][season_dbs['2024-25']['name'] == 'Alex Scott']

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,team_id,name_playerteam,short_name,mng_clean_sheets,mng_draw,mng_goals_scored,mng_loss,mng_underdog_draw,mng_underdog_win,mng_win
0,Alex Scott,MID,Bournemouth,1.6,0,0,11,0,12.8,77,...,3,Bournemouth,BOU,NaN,NaN,NaN,NaN,NaN,NaN,NaN
616,Alex Scott,MID,Bournemouth,0.0,0,0,0,0,0.0,77,...,3,Bournemouth,BOU,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1290,Alex Scott,MID,Bournemouth,0.0,0,0,0,0,0.0,77,...,3,Bournemouth,BOU,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1968,Alex Scott,MID,Bournemouth,0.0,0,0,0,0,0.0,77,...,3,Bournemouth,BOU,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2652,Alex Scott,MID,Bournemouth,0.0,0,0,0,0,0.0,77,...,3,Bournemouth,BOU,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3342,Alex Scott,MID,Bournemouth,0.0,0,0,0,0,0.0,77,...,3,Bournemouth,BOU,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4035,Alex Scott,MID,Bournemouth,0.0,0,0,0,0,0.0,77,...,3,Bournemouth,BOU,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4665,Alex Scott,MID,Bournemouth,0.0,0,0,0,0,0.0,77,...,3,Bournemouth,BOU,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5366,Alex Scott,MID,Bournemouth,0.0,0,0,0,0,0.0,77,...,3,Bournemouth,BOU,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6067,Alex Scott,MID,Bournemouth,0.0,0,0,0,0,0.0,77,...,3,Bournemouth,BOU,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
for season, df in season_dbs.items():
    df["element_id"] = df["name"].str.extract(r'_(\d+)$')
    df["name"] = df["name"].str.replace(r'_\d+$', '', regex=True)

In [ ]:
team_id_to_name = teams.set_index('team_id')['name'].to_dict()
team_id_to_short = teams.set_index('team_id')['short_name'].to_dict()

# Add readable team names to fixtures
fixtures['team_h_name'] = fixtures['team_h'].map(team_id_to_name)
fixtures['team_a_name'] = fixtures['team_a'].map(team_id_to_name)
fixtures['team_h_short'] = fixtures['team_h'].map(team_id_to_short)
fixtures['team_a_short'] = fixtures['team_a'].map(team_id_to_short)

['1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '3',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9']

,code,draw,form,team_id,loss,name,played,points,position,short_name,...,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,NaN,1,0,Arsenal,0,0,2,ARS,...,NaN,False,0,1350,1350,1390,1400,1310,1300,1
1,7,0,NaN,2,0,Aston Villa,0,0,6,AVL,...,NaN,False,0,1145,1240,1130,1180,1160,1300,2
2,91,0,NaN,3,0,Bournemouth,0,0,9,BOU,...,NaN,False,0,1170,1200,1120,1180,1220,1220,127
3,94,0,NaN,4,0,Brentford,0,0,10,BRE,...,NaN,False,0,1130,1180,1100,1100,1160,1260,130
4,36,0,NaN,5,0,Brighton,0,0,8,BHA,...,NaN,False,0,1140,1165,1090,1140,1190,1190,131
5,8,0,NaN,6,0,Chelsea,0,0,4,CHE,...,NaN,False,0,1155,1235,1120,1170,1190,1300,4
6,31,0,NaN,7,0,Crystal Palace,0,0,12,CRY,...,NaN,False,0,1150,1155,1140,1140,1160,1170,6
7,11,0,NaN,8,0,Everton,0,0,13,EVE,...,NaN,False,0,1120,1145,1160,1160,1080,1130,7
8,54,0,NaN,9,0,Fulham,0,0,11,FUL,...,NaN,False,0,1155,1155,1150,1150,1160,1160,34
9,40,0,NaN,10,0,Ipswich,0,0,19,IPS,...,NaN,False,0,1065,1065,1040,1040,1090,1090,8


In [24]:
season_dbs['2018-19'][['fixture','gw']]

,fixture,gw
0,5,1
1,8,1
2,4,1
3,1,1
4,3,1
...,...,...
21785,90,9
21786,88,9
21787,81,9
21788,83,9
